In [ ]:
import json
import numpy as np
import h5py
import matplotlib.pyplot as plt
from transformers import pipeline
from transformers import CLIPModel, CLIPProcessor
import torch
import os
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity 


In [ ]:
# map
with open("../assignment2/FIR-02/data02/database/database_lite.json","r") as f:
    m_idx = json.load(f)
    m_imgs = np.array(m_idx["im_paths"])
    m_loc=np.array(m_idx["loc"])

# query
with open("../assignment2/FIR-02/data02/query/query_lite.json","r") as f:
    q_idx=json.load(f)
    q_imgs=np.array(q_idx["im_paths"])
    q_loc=np.array(q_idx["loc"])
    
# loading the relevance judgements
with h5py.File("../assignment2/FIR-02/data02/london_lite_gt.h5","r") as f:
   fovs = f["fov"][:]
   sim = f["sim"][:].astype(np.uint8)

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
def recall(ranks, pidx, ks):
    recall_at_k = np.zeros(len(ks))
    for qidx in range(ranks.shape[0]):
        for i, k in enumerate(ks):
            if np.sum(np.in1d(ranks[qidx,:k], pidx[qidx])) > 0:
                recall_at_k[i:] += 1
                break

    recall_at_k /= ranks.shape[0]
    return recall_at_k

def apk(pidx, rank, k):
    if len(rank)>k:
        rank = rank[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(rank):
        if p in pidx and p not in rank[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(pidx), k)

def mapk(ranks, pidxs, k):

    return np.mean([apk(a,p,k) for a,p in zip(pidxs, ranks)])

def mapk_many(ranks, pidxs, ks):
    return np.array([mapk(ranks, pidxs, k) for k in ks], dtype=float)

def average_precision(relevant, retrieved):
   precisions = []
   rel = 0
   for i in range(0, len(retrieved)):
      if retrieved[i] in relevant:
         rel += 1
         precisions.append(rel/(i+1))
   return sum(precisions) / len(relevant)

def mean_average_precision(all_relevant, all_retrieved):
   total = 0
   count = 0
   for qid in all_relevant: 
      total += average_precision(all_relevant[qid], all_retrieved.get(qid, []))
      count += 1
   return total / count


def l2_normalize(x, axis=1, eps=1e-12):
   norm = np.linalg.norm(x, axis=axis, keepdims=True)
   return x / (norm + eps)

def get_relevant_images(gt_similarity_matrix, query_idx):
   return np.where(gt_similarity_matrix[query_idx, :] == 1)[0]

def get_retrieved_images(feature_matrix, query_idx):
   return np.argsort(feature_matrix[query_idx])

In [ ]:
model = CLIPModel.from_pretrained("geolocal/StreetCLIP")
processor = CLIPProcessor.from_pretrained("geolocal/StreetCLIP")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
m_feats = np.zeros((len(m_imgs), 768), dtype=np.float32)
for i,img_name in enumerate(m_imgs):
   img = plt.imread(os.path.join('../assignment2/FIR-02/data02/', img_name))
   img = Image.fromarray(img)
   inputs = processor(images=img, return_tensors="pt").to(device)
   with torch.inference_mode():
      image_features = model.get_image_features(**inputs)
   m_feats[i] = image_features.cpu().numpy()

In [ ]:
q_feats = np.zeros((len(m_imgs), 768), dtype=np.float32)
for i,img_name in enumerate(q_imgs):
   img = plt.imread(os.path.join('../assignment2/FIR-02/data02/', img_name))
   img = Image.fromarray(img)
   inputs = processor(images=img, return_tensors="pt").to(device)
   with torch.inference_mode():
      image_features = model.get_image_features(**inputs)
   q_feats[i] = image_features.cpu().numpy()

In [ ]:
_m_feats = l2_normalize(m_feats, axis=1)
_q_feats = l2_normalize(q_feats, axis=1)

similarities = cosine_similarity(_q_feats, _m_feats) 

In [ ]:
all_rel, all_ret = {},{}
for query_idx in range(len(similarities)):
   all_rel[query_idx] = get_relevant_images(sim, query_idx)
   all_ret[query_idx] = get_retrieved_images(similarities,query_idx )

In [ ]:
ranks = np.argsort(-similarities, axis=1)  # (Q, G)

Q = similarities.shape[0]
pidx = [np.array(all_rel[q], dtype=int) for q in range(Q)]

ks = [1, 5, 10, 20]
rec = recall(ranks, pidx, ks)
recall_at_k = recall(ranks, pidx, ks)
mAPs = mapk_many(ranks, pidx, ks)

print(f"MAP : {(mean_average_precision(all_rel, all_ret)*100):.2f}")
for k, r, m in zip(ks, rec, mAPs):
    print(f"Recall@{k}: {r*100:.2f}%   mAP@{k}: {m*100:.2f}%")
